![piggy_bank](piggy_bank.jpg)

<br>

Personal loans are a lucrative revenue stream for banks. The typical interest rate of a two year loan in the United Kingdom is [around 10%](https://www.experian.com/blogs/ask-experian/whats-a-good-interest-rate-for-a-personal-loan/). This might not sound like a lot, but in September 2022 alone UK consumers borrowed [around £1.5 billion](https://www.ukfinance.org.uk/system/files/2022-12/Household%20Finance%20Review%202022%20Q3-%20Final.pdf), which would mean approximately £300 million in interest generated by banks over two years!

You have been asked to work with a bank to clean and store the data they collected as part of a recent marketing campaign, which aimed to get customers to take out a personal loan. They plan to conduct more marketing campaigns going forward so would like you to set up a PostgreSQL database to store this campaign's data, designing the schema in a way that would allow data from future campaigns to be easily imported. 

They have supplied you with a csv file called `"bank_marketing.csv"`, which you will need to clean, reformat, and split, in order to save separate files based on the tables you will create. It is recommended to use `pandas` for these tasks.

Lastly, you will write the SQL code that the bank can execute to create the tables and populate with the data from the csv files. As the bank are quite strict about their security, you'll save SQL files as multiline string variables that they can then use to create the database on their end. 

You have been asked to design a database that will have three tables:

## client

| column | data type | description | original column in dataset |
|--------|-----------|-------------|----------------------------|
| `id` | `serial` | Client ID - primary key | `client_id` |
| `age` | `integer` | Client's age in years | `age` |
| `job` | `text` | Client's type of job | `job` |
| `marital` | `text` | Client's marital status | `marital` | 
| `education` | `text` | Client's level of education | `education` |
| `credit_default` | `boolean` | Whether the client's credit is in default | `credit_default` |
| `housing` | `boolean` | Whether the client has an existing housing loan (mortgage) | `housing` | 
| `loan` | `boolean` | Whether the client has an existing personal loan | `loan` |

<br>

## campaign

| column | data type | description | original column in dataset |
|--------|-----------|-------------|----------------------------|
| `campaign_id` | `serial` | Campaign ID - primary key | N/A - new column |
| `client_id` | `serial` | Client ID - references `id` in the `client` table | `client_id` |
| `number_contacts` | `integer` | Number of contact attempts to the client in the current campaign | `campaign` |
| `contact_duration` | `integer` | Last contact duration in seconds | `duration` |
| `pdays` | `integer` | Number of days since contact in previous campaign (`999` = not previously contacted) | `pdays` |
| `previous_campaign_contacts` | `integer` | Number of contact attempts to the client in the previous campaign | `previous` |
| `previous_outcome` | `boolean` | Outcome of the previous campaign | `poutcome` |
| `campaign_outcome` | `boolean` | Outcome of the current campaign | `y` |
| `last_contact_date` | `date` | Last date the client was contacted | A combination of `day`, `month`, and the newly created `year` |

<br>

## economics

| column | data type | description | original column in dataset |
|--------|-----------|-------------|----------------------------|
| `client_id` | `serial` | Client ID - references `id` in the `client` table | `client_id` |
| `emp_var_rate` | `float` | Employment variation rate (quarterly indicator) | `emp_var_rate` |
| `cons_price_idx` | `float` | Consumer price index (monthly indicator) | `cons_price_idx` |
| `euribor_three_months` | `float` | Euro Interbank Offered Rate (euribor) three month rate (daily indicator) | `euribor3m` |
| `number_employed` | `float` | Number of employees (quarterly indicator)| `nr_employed` |

In [82]:
import pandas as pd
import numpy as np


### Reading in and Splitting the Data


In [83]:
# Read in bank_marketing.csv as a pandas DataFrame.
bank_m = pd.read_csv('bank_marketing.csv')

In [84]:
# inspect the data
bank_m.head()

,client_id,age,job,marital,education,credit_default,housing,loan,contact,month,day,duration,campaign,pdays,previous,poutcome,emp_var_rate,cons_price_idx,cons_conf_idx,euribor3m,nr_employed,y
0,0,56,housemaid,married,basic.4y,no,no,no,telephone,may,13,261,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,1,57,services,married,high.school,unknown,no,no,telephone,may,19,149,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,2,37,services,married,high.school,no,yes,no,telephone,may,23,226,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,3,40,admin.,married,basic.6y,no,no,no,telephone,may,27,151,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,4,56,services,married,high.school,no,no,yes,telephone,may,3,307,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


In [85]:
# define the column names for the new dataframes
client_columns = ['client_id', 'age', 'job', 'marital', 'education', 'credit_default', 'housing', 'loan']
campaign_columns = ['client_id', 'campaign', 'duration', 'pdays', 'previous', 'poutcome', 'y', 'month', 'day']
economic_columns = ['client_id', 'emp_var_rate', 'cons_price_idx', 'euribor3m', 'nr_employed']

# subset the dataframe by the columns
client=bank_m[client_columns]
campaign=bank_m[campaign_columns]
economics=bank_m[economic_columns]

### Data Cleaning

Renaming the columns...

In [86]:
# rename columns in client dataframe
client.rename(columns={'client_id': 'id'}, inplace=True)

# rename columns in the campaign dataframe
campaign.rename(columns={"duration": "contact_duration", 
                         "y": "campaign_outcome", 
                         "campaign": "number_contacts",
                         "previous": "previous_campaign_contacts",
                         "poutcome": "previous_outcome"}, 
                         inplace=True)



# rename columns in the economics dataframe
economics.rename(columns={"euribor3m" : "euribor_three_months", "nr_employed" : "number_employed"}, inplace=True)

Cleaning the columns...

In [87]:
# client table

# clean the "education" column, changing "." to "_" and "unknown" to NumPy's null values.
client['education'] = client['education'].str.replace('.', '_')
client['education'] = client['education'].replace('unknown', np.NaN, inplace=True)

# remove periods from the "job" column
client['job'] = client['job'].str.replace('.', '')


# campaign table

# convert "success" , "failure"  and "nonexistent" in the "previous_outcome"  to binary and numpy's null values
campaign['previous_outcome'] = campaign['previous_outcome'].map({'success': 1, 'failure': 0})
campaign['previous_outcome'].replace('nonexistent', np.NaN, inplace=True)

# convert "success" and "failure" in the "campaign_outcome" column to binary (1 or 0)
campaign['campaign_outcome'] = campaign['campaign_outcome'].map({'success': 1, 'failure': 0})

Creating new columns...

In [88]:
# add a column called campaign_id in campaign, where all rows have a value of 1.
campaign['campaign_id'] = 1

# Add the "year" column with the value 2022 to the 'campaign_data' DataFrame
campaign['year'] = 2022

# Create a datetime column "last_contact_date"
campaign['last_contact_date'] = pd.to_datetime(
                                campaign['year'].astype(str) + '-' +                                                       campaign['month'].astype(str) + '-' +                                                     campaign['day'].astype(str), errors='coerce'
)

Deleting columns...

In [89]:
# remove "day", "month", and "year" columns that were used to create the "last_contact_date" column
campaign = campaign.drop(['day', 'month', 'year'], axis=1)

### Saving the data

Storing the DataFrames...

In [90]:
# client data
client.to_csv('client.csv', index=False)

# campaign data
campaign.to_csv('campaign.csv', index=False)

# economics data
economics.to_csv('economics.csv', index=False)

### Designing the database

In [91]:
# Create the client table and specify the SQL code as a multi-line string
client_table="""
CREATE TABLE client(
id SERIAL PRIMARY KEY,
age INT,
job TEXT,
marital TEXT, 
education TEXT,
credit_default BOOLEAN, 
housing BOOLEAN,
loan BOOLEAN);

--copy data from client file into db
\copy client FROM 'client.csv' DELIMITER ',' CSV HEADER;
"""


# Create the campaign table and specify the SQL code as a multi-line string
campaign_table="""
CREATE TABLE campaign(
campaign_id SERIAL PRIMARY KEY,
client_id SERIAL REFERENCES client(id),
number_contacts INT,
contact_duration INT, 
pdays INT,
previous_campaign_contacts INT,
campaign_outcome BOOLEAN,
campaign_id BOOLEAN,
last_contact_date DATE);

--copy data from
\copy campaign FROM 'campaign.csv' DELIMITER ',' CSV HEADER;
"""


# Create the economics table and specify the SQL code as a multi-line string
economics_table="""
CREATE TABLE economics(
client_id SERIAL REFERENCE client(id),
emp_var_rate FLOAT,
cons_price_idx FLOAT,
euribor_three_months FLOAT,
number_employed FLOAT
);

--copy data from economics file
\copy campaign FROM 'economics.csv' DELIMITER ',' CSV HEADER;
"""